In [1]:
# 모듈, 경로 설정
import pandas as pd
import pydeck as pdk
import geopandas as gpda
import config
import json
import requests
import sys
import folium

path = "C:\github/aide-analysis/"
sys.path.append(path)

from src.sql import DatabaseHandler
db = DatabaseHandler()

from core.config import Config
config = Config()


In [ ]:
# Vworld 시군구 경계 API 호출
def gl(sig_cd):
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 행정동 경계
    data = "LT_C_ADSIGG_INFO"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f"sig_cd:like:{11}"
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"

    # 요청 결과
    res = json.loads(requests.get(url).text)

    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current == page_total:
            break
    featureCollection["features"] = features
    geo = featureCollection
    return geo
geo = gl(11)

In [ ]:
# Vworld 시군구 경계 API 호출
def gl2(sig_cd):
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 행정동 경계
    data = "LT_C_ADSIGG_INFO"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f"sig_cd:=:{11650}"
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"

    # 요청 결과
    res = json.loads(requests.get(url).text)

    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current == page_total:
            break
    featureCollection["features"] = features
    geo = featureCollection
    return geo
geo2 = gl2(11)

In [ ]:
# params
center = [37.541, 126.986]
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    width = 750,
    height = 500,
    tiles = tiles[2]
)


# Marker
"""
location : 마커를 추가할 경위도 좌표/ 
popup : 표기할 팝업문구(클릭시 문구)/ 
tooltip : 표기할 툴팁 지정(마우스 오버))
"""
folium.Marker(
    location = [lat, lon],
    popup = '반포자이',
    tooltip = '반포자이',
    icon = folium.Icon('red', icon = icon[0])
).add_to(m)

# GeoJson 활용
folium.GeoJson(
    geo,
    name = '서초구',
    style_function = lambda x: {'fillColor':'#00000000' ,'color':'black','weight' :'1'}
).add_to(m)


folium.GeoJson(
    geo2,
    name = '서초구',
    style_function = lambda x: {'fillColor':'red' ,'color':'black','weight' :'1','fillOpacity':'0.3'}
).add_to(m)

m
# m.save('data/result.html')

### DB연결후 적용

#### 세대수

In [ ]:
# 데이터 불러오기
sql = f"""
SELECT hdongName, 
       totalSaedae 
FROM aide.population_saedae
WHERE sidoCode ='11' 
      and sigunguCode != '11000' 
      and date = (SELECT MAX(date) 
                  FROM aide.population_saedae);
"""
df = db.read_table(sql)

In [ ]:

# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[2]
)

# Marker
folium.Marker(
    location = [37.5146321, 127.018293],
    popup = '회사',
    tooltip = '회사',
    icon = folium.Icon('red', icon = icon[0])
).add_to(m)


# Marker
folium.Marker(
    location = [37.4857983, 126.948162],
    popup = '집',
    tooltip = '집',
    icon = folium.Icon('red', icon = icon[0])
).add_to(m)


# Polygon
m.choropleth(
    geo_data = geo,
    data = df,
    columns = ('hdongName','totalSaedae'),
    key_on = 'feature.properties.full_nm',
    fill_color = 'YlOrRd',
    fill_opacity = 0.5,
    line_opacity = 0.2
)
m

In [ ]:
location_data = [[37.5146321, 127.018293],
                [37.4857983, 126.948162]]
folium.PolyLine(locations = location_data, tooltip = "Polyline").add_to(m)
m

### 연속지적도 + 아파트 전세 공급평단가(최근날짜기준)

In [ ]:
# 최근실거래가 기준 아파트 공급평단가

sigunguCode = 41463
sql = f"""
SELECT T1.pnuCode,
	   T1.contractYear,
       T1.contractMonth,
       T1.tradingSupplyUnitPriceAverage,
       T1.jeonseSupplyUnitPriceAverage,
       T1.wolseSupplyUnitPriceAverage,
       T1.rn
FROM(
    -- 단지별 년,월별 평균거래가의 평균
    SELECT pnuCode, 
           contractYear, 
           contractMonth,
           AVG(tradingSupplyUnitPriceAverage) as tradingSupplyUnitPriceAverage,
           AVG(jeonseSupplyUnitPriceAverage) AS jeonseSupplyUnitPriceAverage,
           AVG(wolseSupplyUnitPriceAverage) AS wolseSupplyUnitPriceAverage,
           row_number() OVER(PARTITION BY pnuCode ORDER BY contractYear DESC) AS rn

    FROM aide.apartment_danji_monthly_price
    WHERE pnuCode LIKE '{sigunguCode}%'
    GROUP BY pnuCode, contractYear, contractMonth 

    ) AS T1
WHERE T1.rn=  1
"""
price_df = db.read_table(sql)

In [ ]:
# 전세가 공급평단가를 기준으로 측정하기 위해 결측값제거
price_df = price_df.dropna(subset = 'jeonseSupplyUnitPriceAverage')
price_df = price_df[['pnuCode','contractYear','contractMonth','jeonseSupplyUnitPriceAverage']]
price_df

In [ ]:
# API를 호출할 수 있는 형태로 변형
lst = ",".join(price_df['pnuCode'])

In [ ]:
def continue_land():
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 연속지적도
    data = "LP_PA_CBND_BUBUN"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f'pnu:IN:{lst}'
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"

    # 요청 결과
    res = json.loads(requests.get(url).text)

    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current== page_total:
            break 
    featureCollection["features"] = features
    geo = featureCollection
    return geo

geo = continue_land()

In [ ]:

# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[0]
)


# Polygon
m.choropleth(
    geo_data = geo,
    data = price_df,
    columns = ('pnuCode','jeonseSupplyUnitPriceAverage'),
    key_on = 'feature.properties.pnu',
    fill_color = 'YlOrRd',
    fill_opacity = 0.5,
    line_opacity = 0.2
)
m

시군구별 전세 전용 평단가 평균

In [ ]:
sql33 = f"""
SELECT T1.sigunguCode, T1.contractYear, T1.contractMonth, T1.tradingJeonyongUnitPriceAverage, T1.jeonseJeonyongUnitPriceAverage, T1.wolseJeonyongUnitPriceAverage, T1.rn
FROM(
SELECT sigunguCode, 
		contractYear, 
		contractMonth, 
		AVG(tradingJeonyongUnitPriceAverage) AS tradingJeonyongUnitPriceAverage , 
		AVG(jeonseJeonyongUnitPriceAverage) AS jeonseJeonyongUnitPriceAverage, 
		AVG(wolseJeonyongUnitPriceAverage) AS wolseJeonyongUnitPriceAverage,
		row_number() OVER(PARTITION BY sigunguCode ORDER BY contractYear DESC) AS rn
FROM aide.apartment_danji_monthly_price
GROUP BY sigunguCode, contractYear, contractMonth
) AS T1
WHERE T1.rn = 1
"""
test = db.read_table(sql33)

In [ ]:
test[test['sigunguCode']=="42810"]

In [ ]:
# join
lst = ",".join(test['sigunguCode'])

In [ ]:
# Vworld 시군구 경계 API 호출
def gl(sig_cd):
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 행정동 경계
    data = "LT_C_ADSIGG_INFO"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f"sig_cd:IN:{lst}"
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"

    # 요청 결과
    res = json.loads(requests.get(url).text)

    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current == page_total:
            break
    featureCollection["features"] = features
    geo = featureCollection
    return geo
geo = gl(11)

In [ ]:

# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[0]
)


# Polygon
m.choropleth(
    geo_data = geo,
    data = test,
    columns = ('sigunguCode','jeonseJeonyongUnitPriceAverage'),
    key_on = 'feature.properties.sig_cd',
    fill_color = 'YlOrRd',
    fill_opacity = 0.5,
    line_opacity = 0.2
)
m

- 최근에 거래가 전세거래가 아닌 거래(매매 혹은 월세)가 있던 지역은 회색
- 현재는 가장 최근 거래에 따라서 조회를 하기 때문에 매매, 전세, 혹은 월세 중 없는 거래가 있을 수 있으므로 특정 거래방식의 선택에따라 없는 부분이 생기지만 향후 필요한 거래형태에 따라 특정기간내 특정형태거래를 조회하는 쿼리를 만들면 모든 시군구를 채울 수 있음 

### 시군구경계 + 미분양 시각화

#### sig_cd 로 API를 호출하게 되면 시군구 뎁스가 맞지않아 모든 시군구가 매핑이 되지않고, full_nm으로 API를 호출하면 "JSONDecodeError: Expecting value: line 1 column 1 (char 0)" 에러가 발생한다. 일시 중단

In [ ]:
sql = f"""

SELECT * 
FROM aide.unsold_sigungu
WHERE year = '2022' 
      AND month = '03'
      AND sigunguName != '계'
      
"""
df = db.read_table(sql)
df

In [ ]:
lst = ','.join(df['sigunguCode'])

In [ ]:
# Vworld 시군구 경계 API 호출
import xmltodict
def gl():
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 행정동 경계
    data = "LT_C_ADSIGG_INFO"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f"sig_cd:LIKE:{11}%"
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"


    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        
        # res = xmltodict.parse(requests.get(url).text)
        # res = json.loads(json.dumps(res))
        
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current == page_total:
            break
    featureCollection["features"] = features
    geo = featureCollection
    return geo
geo = gl()

In [ ]:
# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[0]
)


# Polygon
m.choropleth(
    geo_data = geo,
    data = df,
    columns = ('sigunguCode','unsold'),
    key_on = 'feature.properties.sig_cd',
    fill_color = 'YlGnBu',
    fill_opacity = 0.5,
    line_opacity = 0.2
)
m

### 입력한 지번 반경에 있는 비슷한 스펙의 매물을 조회 및 건물 지하철등 상업시설조회 

**구현하고자 하는 기능:**
1. ~~지번을 입력받으면 해당 지번 물건의 반경 안에 있는 매물들을 모두 지도상에 표시한다~~
2. ~~매물들의 세부정보 popup에 추가~~
3. 해당 지번은 과 비슷한 스펙의 주변 매물들이 지도상에 표시가 된다.
4. ~~마우스를 올리면 매물에 대한 정보가 요약된다. ~~
5. ~~주변 지하철이 표시가 된다.~~
6. 주변 공원 및 큰 상점들이 표시가 된다.

In [22]:
# 경로설정
import sys
path = "C:\github/aide-analysis/"
sys.path.append(path)

# 모듈 import
import pandas as pd
import src
import folium
from src.openapi import KakaoLocalAPI as kakao
from openpyxl import load_workbook
from src.utils import insert_dataframe_to_excel_sheet
from src.sql import DatabaseHandler
from src.temp import *


db = DatabaseHandler()

# 카카오 API 호출
docs = kakao.search_address('서울시 서초구 반포동 20-43')


# 파싱정보
# 지번 추출
jibeon = docs['documents'][0]['address_name']

# 시도/ 시군구/ 행정동 코드 추출
sidoCode = docs["documents"][0]["address"]["h_code"][:2]
sigunguCode = docs["documents"][0]["address"]["h_code"][:5]
hdongCode = docs["documents"][0]["address"]["h_code"]

# 행정구역 명칭
sido_ = docs["documents"][0]["address"]["region_1depth_name"]
sigungu_ = docs["documents"][0]["address"]["region_2depth_name"]
if sigungu_ == "":
    sigungu_ = sido_
hdong_ = docs['documents'][0]['address']['region_3depth_h_name']
if hdong_ == "":
    hdong_ = docs['documents'][0]['address']['region_3depth_name']

# 경도, 위도
lon = docs['documents'][0]['address']['x'] 
lat = docs['documents'][0]['address']['y'] 
radius = 1000

# 단지명
danjiName = docs['documents'][0]['road_address']['building_name']

# 시계열 인자
startyear = datetime.now().year-10
endyear = datetime.now().year-1



In [23]:
# folium용
# 입력된지번의 일정 반경내 아파트와 아파트 정보
sql = f"""
WITH tmp AS (  SELECT A.pnuCode, 
                      addressJibeon, 
                      addressRoad, 
                      danjiName, 
                      usePermissionDate, 
                      B.contractYear, 
                      B.contractMonth, 
                      B.jeonseJeonyongUnitPriceAverage, 
                      buildingAreaPerLandArea, 
                      floorAreaRatio, 
                      mainPurposeCodeName, 
                      otherPurpose, 
                      saedaeCount, 
                      totalParkingLotCount, 
                      lon, 
                      lat, 
                      coord, 
                      geom
               FROM aide.apartment_danji_information AS A
               JOIN (SELECT T1.*
                     FROM(
                         SELECT pnuCode,
                                contractYear,
                                contractMonth,
                                supplyArea,
                                tradingJeonyongUnitPriceAverage,
                                jeonseJeonyongUnitPriceAverage,
                                wolseJeonyongUnitPriceAverage,
                                ROW_NUMBER() OVER(PARTITION BY pnuCode  ORDER BY contractYear DESC, contractMonth DESC) AS num
                         FROM aide.apartment_danji_monthly_area_price
                         WHERE pnuCode LIKE '{sidoCode}%' AND jeonseJeonyongUnitPriceAverage is not null
                        ) AS T1
                     WHERE T1.num = 1 
                   ) AS B
                   on A.pnuCode = B.pnuCode 
                   
)SELECT tmp.*,
        ST_Distance_Sphere(POINT({lon},{lat}),geom) AS distance
FROM tmp
WHERE ST_Distance_Sphere(POINT({lon},{lat}), geom) <={radius}
ORDER BY distance ASC;
"""
df= db.read_table(sql)


# 반경내 지하철역 정보 테이블
sql2 = f"""
SELECT * , 
       ST_Distance_Sphere(POINT({lon},{lat}),POINT(lon,lat)) AS distance
FROM aide.railway_station_info
WHERE ST_Distance_Sphere(POINT({lon},{lat}),POINT(lon,lat)) <= {radius}
ORDER BY distance ASC;
"""
train = db.read_table(sql2)

In [26]:
df

,pnuCode,addressJibeon,addressRoad,danjiName,usePermissionDate,contractYear,contractMonth,jeonseJeonyongUnitPriceAverage,buildingAreaPerLandArea,floorAreaRatio,mainPurposeCodeName,otherPurpose,saedaeCount,totalParkingLotCount,lon,lat,coord,geom,distance
0,1165010600100670002,서울특별시 서초구 잠원동 67-2번지,서울특별시 서초구 잠원로 46-38,브라운스톤잠원,20030925,2021,12,3677.851356,25,297,공동주택,공동주택(아파트),83,94,127.009163,37.508170,POINT(127.009162726394 37.5081699251029),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\x15\x8d...,261.025417
1,1165010600100590003,서울특별시 서초구 잠원동 59-3번지,서울특별시 서초구 신반포로 261,잠원현대훼밀리,19970827,2022,3,3607.723859,25,342,공동주택,공동주택,113,113,127.012445,37.508982,POINT(127.012444652076 37.5089823037232),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00Lk\xa7\x...,317.841384
2,1165010600100670001,서울특별시 서초구 잠원동 67-1번지,서울특별시 서초구 잠원로 46-4,데뜨아르,20080422,2022,4,4281.096817,24,211,공동주택,공동주택,22,26,127.008663,37.508635,POINT(127.008663293929 37.5086345196832),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00r\x06}\x...,328.906576
3,1165010600101570000,서울특별시 서초구 잠원동 157번지,서울특별시 서초구 신반포로33길 15,동아,,2022,4,3893.281267,23,316,공동주택,공동주택(아파트),991,1025,127.010929,37.509316,POINT(127.010929282174 37.5093163798427),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00d`\xbb\x...,329.775004
4,1165010700100200043,서울특별시 서초구 반포동 20-43번지,서울특별시 서초구 신반포로 270,반포자이,20090313,2022,4,4596.193480,14,270,공동주택,공동주택(아파트),3410,6075,127.013900,37.508401,POINT(127.013900193294 37.5084007014215),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\xc6\xe6...,340.285957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,1165010600100760002,서울특별시 서초구 잠원동 76-2번지,서울특별시 서초구 반포대로 310,두산위브,20030625,2021,12,3548.702242,36,299,공동주택,공동주택(아파트),49,49,127.000597,37.506646,POINT(127.000597344152 37.5066461134614),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00:fq\xc9\...,921.244489
71,1165010600101590000,서울특별시 서초구 잠원동 159번지,서울특별시 서초구 잠원로 117,아크로리버뷰신반포,20180604,2022,4,6737.905985,16,276,공동주택,공동주택(아파트),595,890,127.007283,37.514498,POINT(127.007283232196 37.5144979558146),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\xd3\x05...,964.352625
72,1165010600100630000,서울특별시 서초구 잠원동 63번지,서울특별시 서초구 잠원로 130,CJ빌리지,20050701,2021,10,4654.878415,30,250,공동주택,공동주택(아파트),17,29,127.008642,37.514897,POINT(127.008642332057 37.5148972246205),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\xf1b\x9...,973.351894
73,1165010800113010006,서울특별시 서초구 서초동 1301-6번지,서울특별시 서초구 서운로 221,래미안서초스위트,20090430,2022,4,4418.968058,13,279,공동주택,아파트,392,505,127.021226,37.502845,POINT(127.021226446301 37.5028447028925),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\x14++\x...,979.967854


In [25]:
from IPython.display import IFrame
# params
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']
center = [37.5150660594, 127.0180350423]

# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 14,
    tiles = tiles[2]
)


text=f"""지번:{df.iloc[i]['addressJibeon']} <br> 
      단지명 :{df.iloc[i]['danjiName']} <br> 
      사용승인일 :{df.iloc[i]['usePermissionDate']} <br> 
      전세전용평단가평균 : {df.iloc[i]['jeonseJeonyongUnitPriceAverage']} <br> 
      용적률 : {df.iloc[i]['floorAreaRatio']} <br>
      주용도 :{df.iloc[i]['mainPurposeCodeName']} <br>
      기타용도 : {df.iloc[i]['otherPurpose']} <br>
      세대수 : {df.iloc[i]['saedaeCount']} <br>
      총 주차대수 : {df.iloc[i]['totalParkingLotCount']} <br>
      거리 : {df.iloc[i]['distance']}""" 


text2 = f"""지번:{df.iloc[0]['addressJibeon']} <br> 
          단지명 :{df.iloc[0]['danjiName']} <br> 
          사용승인일 :{df.iloc[0]['usePermissionDate']} <br> 
          전세전용평단가평균 : {df.iloc[0]['jeonseJeonyongUnitPriceAverage']} <br> 
          용적률 : {df.iloc[0]['floorAreaRatio']} <br>
          주용도 :{df.iloc[0]['mainPurposeCodeName']} <br>
          기타용도 : {df.iloc[0]['otherPurpose']} <br>
          세대수 : {df.iloc[0]['saedaeCount']} <br>
          총 주차대수 : {df.iloc[0]['totalParkingLotCount']} <br>
          거리 : {df.iloc[0]['distance']}""" 


iframe = folium.IFrame(text)
iframe2 = folium.IFrame(text2)

# 입력받은 지번을 제외한 매물 마킹
for i in range(1,len(df)):
    folium.Marker(
        location = [df.iloc[i]['lat'], df.iloc[i]['lon']],      
        popup = folium.Popup(iframe, min_width =400, max_width = 400),
        tooltip = df.iloc[i]['danjiName'],
        icon = folium.Icon('blue', icon = icon[1])).add_to(m)
    
# 입력받은 지번 매물 마킹
folium.Marker(
    location = [lat,lon],
    popup = folium.Popup(iframe2, min_width = 500, max_widht = 500),
    tooltip = danjiName,
    icon = folium.Icon('red', icon = icon[0])).add_to(m)
 
    
for i in range(0, len(train)):
    folium.Marker(
        location = [train.iloc[i]['lat'], train.iloc[i]['lon']],
        popup = train.iloc[i]['lineName'],
        tooltip = train.iloc[i]['lineName'],
        icon = folium.Icon('green', icon = icon[1])).add_to(m)
m    


### pydeck 활용

## 추후 해야할것.
- 목적에 맞게 전달력있는 시각화방법
- 파라미터 수정을 통한 원하는 정보전달방법 
- 바로 아래 셀에서 좌표가 없는 아파트가 존재하는데 이유 찾기
- 좀더 랜더링을 빠르게할 수 없을지 생각해보기

In [ ]:
# pydeck 연습용


# 입력된지번의 일정 반경내 아파트와 아파트 정보
sql = f"""
SELECT A.pnuCode, addressJibeon, addressRoad, danjiName, usePermissionDate, B.contractYear, B.contractMonth, B.jeonyongArea, B.jeonseJeonyongUnitPriceAverage, buildingAreaPerLandArea, floorAreaRatio, mainPurposeCodeName, otherPurpose, saedaeCount, totalParkingLotCount, lon, lat, coord, geom
FROM aide.apartment_danji_information AS A
JOIN (SELECT T1.*
    FROM(
         SELECT pnuCode,
                 contractYear,
                 contractMonth,
                 jeonyongArea,
                 supplyArea,
                 tradingJeonyongUnitPriceAverage,
                 jeonseJeonyongUnitPriceAverage,
                 wolseJeonyongUnitPriceAverage,
                 ROW_NUMBER() OVER(PARTITION BY pnuCode ,jeonyongArea ORDER BY contractYear DESC, contractMonth DESC) AS num
         FROM aide.apartment_danji_monthly_area_price
         WHERE pnuCode LIKE '{sidoCode}%' AND jeonseJeonyongUnitPriceAverage is not null
        ) AS T1
        WHERE T1.num = 1 
) AS B
on A.pnuCode = B.pnuCode 

"""
df= db.read_table(sql)


In [ ]:
len(df[df['lon'].isna()])

In [ ]:
df[df['lon'].isna()]

In [ ]:
df = df.drop(index = df[df['lon'].isna()].index)

In [ ]:
# import pydeck
import pydeck as pdk

#  wkt to shapely geometry
from shapely import wkt
df['coord'] = df['coord'].apply(wkt.loads)

In [ ]:
# pydeck 을 사용하기 위해 리스트 형태로 변환
from tqdm import trange, notebook

# point to list
for i in notebook.tqdm(range(0, len(df))):
    x, y  = df['coord'].iloc[i].xy
    df['coord'].iloc[i] = [[x[0], y[0]]]
    
# # polygon to list
# for i in range(0, len(poly)):
#     poly['geometry'].iloc[i] = list(poly['geometry'].iloc[i].exterior.coords.xy)

In [ ]:
# 방해되는 컬럼 제거
df = df.drop(columns = 'geom')

In [ ]:
# df = pd.read_json(df.to_json())

In [ ]:
df['정규화평당가'] = df['jeonseJeonyongUnitPriceAverage'] / df['jeonseJeonyongUnitPriceAverage'].max()

In [ ]:
# scatterplotLayer
# Make layer
center = [df.iloc[0]['lat'], df.iloc[0]['lon']]
layer = pdk.Layer(
    'ScatterplotLayer', 
    df, 
    get_position='[lon, lat]',
    get_radius = 50,
    get_fill_color='[250*정규화평당가, 75*정규화평당가, 255]', # Set an RGBA value for fill
    pickable=True, # plot세부정보 확인
    auto_highlight=False
)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=127.0180350423,
    latitude=37.5150660594,
    zoom=12)


# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
# heatmapLayer
layer = pdk.Layer(
    'HeatmapLayer',
    df,
    get_position='[lon, lat]',
    opacity=0.3# 불투명
)


view_state = pdk.ViewState(
    longitude=127.0180350423,
    latitude=37.5150660594,
    zoom=10)

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
layer = pdk.Layer(
    'CPUGridLayer', # 대용량 데이터의 경우 'GPUGridLayer'
    df,
    get_position='[lon, lat]',
    pickable=True,
    auto_highlight=True
)


view_state = pdk.ViewState(
    longitude=127.0180350423,
    latitude=37.5150660594,
    zoom=10)



r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
layer = pdk.Layer(
    'CPUGridLayer', # 대용량 데이터의 경우 'GPUGridLayer'
    df,
    get_position='[lon, lat]',
    pickable=True,
    auto_highlight=True,
    elevation_scale = 50,
    elevation_range=[0, 200],
     extruded=True
)


view_state = pdk.ViewState(
    longitude=127.0180350423,
    latitude=37.5150660594,
    zoom=10)


# layer.extruded = True;
# layer.get_elevation = 'saedaeCount';
# layer.elevation_scale = 50

view_state.bearing=15
view_state.pitch=45

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
# Define a layer to display on a map
layer = pdk.Layer(
    "HexagonLayer",
    df,
    get_position='[lon, lat]',
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0,200],
    extruded=True,
    coverage=1,
)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=127.0180350423, latitude=37.5150660594, zoom=10, min_zoom=5, max_zoom=15, pitch=20, bearing=-27.36,
)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html("hexagon_layer.html")

In [ ]:
# pydeck POLYGON

# data load
sql3 = f"""
SELECT * FROM aide.polygon_bdong
WHERE bdongCode lIKE '11650%';
"""
poly = db.read_table(sql3)

#  wkt to shapely Polygon
from shapely import wkt
poly['geometry'] = poly['geometry'].apply(wkt.loads)


# polygon to list
for i in range(0, len(poly)):
    poly['geometry'].iloc[i] = list(poly['geometry'].iloc[i].exterior.coords.xy)[0]
    
# 임의의 컬럼생성
poly['정규화인구'] = [0.3213213,0.32132140, 0.12312516,0.6786547,0.645645,0.512534,0.3451346,0.754673456,0.413412,0.15153]

In [ ]:
# pydeck POLYGON

# data load
sql3 = f"""
SELECT * FROM aide.polygon_bdong
WHERE bdongCode lIKE '11650%';
"""
poly = db.read_table(sql3)

In [ ]:
#  wkt to shapely Polygon
from shapely import wkt
poly['geometry'] = poly['geometry'].apply(wkt.loads)
# 임의의 컬럼생성
poly['정규화인구'] = [0.3213213,0.32132140, 0.12312516,0.6786547,0.645645,0.512534,0.3451346,0.754673456,0.413412,0.15153]

In [ ]:
for i in range(0, len(poly)):
    poly['geometry'].iloc[i] = list(poly['geometry'].iloc[i].exterior.coords)
    

In [ ]:
poly

In [ ]:

# Make layer
layer = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    poly, # 시각화에 쓰일 데이터프레임
    get_polygon='geometry', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[255*정규화인구, 100*정규화인구, 255]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
    pickable=True, # 지도와 interactive 한 동작 on
    auto_highlight=True # 마우스 오버(hover) 시 박스 출력
)

# Set the viewport location
center = [126.986, 37.565]
view_state = pdk.ViewState(
    longitude=center[0],
    latitude=center[1],
    zoom=10)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

## 전입전출 이동 시각화

In [ ]:
sql =f"""
SELECT DISTINCT transfer.inHdongCode,
       st_x(st_centroid(ST_GeomFromText(hdong.geometry))) as x,
	   st_y(st_centroid(ST_GeomFromText(hdong.geometry))) as y
FROM aide.polygon_hdong AS hdong
RIGHT JOIN (SELECT inHdongCode
			FROM aide.population_transfer
			WHERE inYear = 2020 AND inHdongCode LIKE '11%' AND outHdongCode LIKE '11%') AS transfer
            ON hdong.hdongCode = transfer.inHdongCode;
"""
inTransaction = db.read_table(sql)

In [ ]:
sql2 = f"""
SELECT DISTINCT transfer2.outHdongCode,
       st_x(st_centroid(ST_GeomFromText(hdong.geometry))) as x,
	   st_y(st_centroid(ST_GeomFromText(hdong.geometry))) as y
FROM aide.polygon_hdong AS hdong
RIGHT JOIN (SELECT outHdongCode
			FROM aide.population_transfer
            WHERE inYear = 2020 AND outHdongCode LIKE '11%' AND outHdongCode LIKE '11%') AS transfer2
            ON hdong.hdongCode = transfer2.outHdongCode;
"""
outTransaction = db.read_table(sql2)

In [ ]:
df = pd.concat([inTransaction, outTransaction],axis =1)
df

In [ ]:
df.columns = ['inHdongCode','in_x','in_y','outHdongCode','out_x','out_y']

In [ ]:
df

In [ ]:
# import pydeck
import pydeck as pdk

In [ ]:
layer = pdk.Layer(
    'ArcLayer',
    df,
    get_source_position='[in_x, in_y]',
    get_target_position='[out_x, out_y]',
    get_width='10',
    get_source_color='[255, 255, 120]',
    get_target_color='[255, 0, 0]',
    pickable=True,
    auto_highlight=True
)

# pydeck.data_utils.compute_view 는 Points 들의 경도, 위도를 리스트로 주면, 알아서 view_state 를 만들어줍니다.
view_state = pdk.data_utils.compute_view(df[['in_x', 'in_y']].values)
view_state.zoom = 12
view_state.bearing = -15
view_state.pitch = 45

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
sql3 = f'''

SELECT geometry FROM aide.polygon_hdong;
'''
ls = db.read_table(sql3)
ls

In [ ]:
ls['geometry'][0]

In [ ]:
sql = f"""
SELECT geometry 
FROM aide.polygon_bdong;

"""
tmp = db.read_table(sql)

In [ ]:
tmp['geometry'][0].wkt.loads

### 시군구경계 + 미분양